In [ ]:
!pip install seldon-deploy-sdk
!pip install alibi
!pip install alibi-detect

In [ ]:
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn import neighbors, linear_model, svm, tree, ensemble
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import pickle 
import joblib
import dill
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense, InputLayer
from alibi.explainers import AnchorTabular
from alibi_detect.datasets import fetch_kdd
from alibi_detect.models.tensorflow.losses import elbo
from alibi_detect.od import OutlierVAE
from alibi_detect.cd import TabularDrift
from alibi_detect.utils.data import create_outlier_batch
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_tabular, plot_roc
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

##Model Training

In this section we will adjust a classifier that will classify consumers in the different client categories. 

In [ ]:
class Class_Fit(object):
    def __init__(self, clf, params=None):
        if params:            
            self.clf = clf(**params)
        else:
            self.clf = clf()

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def grid_search(self, parameters, Kfold):
        self.grid = GridSearchCV(estimator = self.clf, param_grid = parameters, cv = Kfold)
        
    def grid_fit(self, X, Y):
        self.grid.fit(X, Y)
        
    def grid_predict(self, X, Y):
        self.predictions = self.grid.predict(X)
        print("Precision: {:.2f} % ".format(100*metrics.accuracy_score(Y, self.predictions)))
       

Mean refers to the average amount spent within each transaction. Each category represents a different category of products and was derived through k-means clustering.

In [ ]:
columns = ['mean', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4' ] # change to feature_names?
class_names = ['0','1','2','3','4','5','6','7','8','9','10']
X_train = np.load('data/X_train.npy')
Y_train = np.load('data/Y_train.npy')
X_test = np.load('data/X_test.npy')
Y_test = np.load('data/Y_test.npy')

## Support Vector Machine 

In [ ]:
svc = Class_Fit(clf = svm.LinearSVC)
svc.grid_search(parameters = [{'C':np.logspace(-2,2,10)}], Kfold = 5)

In [ ]:
svc.grid_fit(X = X_train, Y = Y_train)

In [ ]:
svc.grid_predict(X_test, Y_test)

## Logistic Regression

In [ ]:
lr = Class_Fit(clf = linear_model.LogisticRegression)
lr.grid_search(parameters = [{'C':np.logspace(-2,2,20)}], Kfold = 5)
lr.grid_fit(X = X_train, Y = Y_train)

In [ ]:
lr.grid_predict(X_test, Y_test)

In [ ]:
lr.grid.best_estimator_.predict_proba([[169.44666667,  19.82924814,  22.28823229,  28.95207932, 23.49411811,   5.43632215]])

In [ ]:
models = ['lr','svc']

for model in models:
  if not os.path.exists('models/' + model):
    os.mkdir('models/' + model)

In [ ]:
filename = 'models/lr/model.joblib'
joblib.dump(lr.grid.best_estimator_, filename)

filename = 'models/svc/model.joblib'
joblib.dump(svc.grid.best_estimator_, filename)

## Push model artefacts to GCP

In [ ]:
# Push artefact to GCP
# !gsutil cp model/<.sav model file> gs://tom-seldon-examples/datareply-workshop/<YOUR NAME>/<MODEL TYPE>/model.sav
!gsutil cp models/lr/model.joblib gs://tom-seldon-examples/datareply-workshop/sean/lr/model.joblib

In [ ]:
!gsutil cp models/svc/model.joblib gs://tom-seldon-examples/datareply-workshop/sean/svc/model.joblib

## Deploy models to Seldon

We can now deploy our model to the dedicated Seldon Deploy cluster which we have configured for this workshop. To do so we will interact with the Seldon Deploy SDK and deploy our model using that.

First, setting up the configuration and authentication required to access the cluster. Make sure to fill in the SD_IP variable to be the same as the cluster you are using.

In [ ]:
SD_IP = "139.59.203.129"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate("admin@seldon.io", "12341234")
    api_client = ApiClient(config)
    return api_client

In [ ]:
DEPLOYMENT_NAME = "customer-lr"
NAMESPACE = "test"
MODEL_LOCATION = "gs://tom-seldon-examples/datareply-workshop/sean/lr"

PREPACKAGED_SERVER = "SKLEARN_SERVER"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

## Explainer

Next, we shall train an explainer to glean deeper insights into the decisions being made by our model. 

In [ ]:
clf = lr.grid.best_estimator_

In [ ]:
predict_fn = lambda x: clf.predict(x)

In [ ]:
explainer = AnchorTabular(predict_fn, columns)

In [ ]:
explainer.fit(X_train, disc_perc=(25, 50, 75))

In [ ]:
idx = 0
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

In [ ]:
explanation = explainer.explain(X_test[idx], threshold=0.9)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

In [ ]:
with open("models/lr/explainer.dill", "wb") as model_f:
        dill.dump(explainer, model_f)

In [ ]:
!python -V

In [ ]:
!gsutil cp models/lr/explainer.dill gs://tom-seldon-examples/datareply-workshop/sean/lr/explainer.dill

## Deployment

We can now deploy our explainer alongside our model. First we define the explainer configuration. 

In [ ]:
EXPLAINER_TYPE = "AnchorTabular"
EXPLAINER_URI = "gs://tom-seldon-examples/datareply-workshop/sean/lr"

explainer_spec = {
                    "type": EXPLAINER_TYPE,
                    "modelUri": EXPLAINER_URI,
                    "containerSpec": {
                        "name": "",
                        "resources": {}
                    }
                }

In [ ]:
mldeployment['spec']['predictors'][0]['explainer'] = explainer_spec
mldeployment

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

## Outlier Detection 


## Method

The Variational Auto-Encoder ([VAE](https://arxiv.org/abs/1312.6114)) outlier detector is first trained on a batch of unlabeled, but normal (*inlier*) data. Unsupervised training is desireable since labeled data is often scarce. The VAE detector tries to reconstruct the input it receives. If the input data cannot be reconstructed well, the reconstruction error is high and the data can be flagged as an outlier. The reconstruction error is either measured as the mean squared error (MSE) between the input and the reconstructed instance or as the probability that both the input and the reconstructed instance are generated by the same process.

In [ ]:
normal_batch = create_outlier_batch(X_train, Y_train, n_samples=2886, perc_outlier=0)
X_train, y_train = normal_batch.data.astype('float'), normal_batch.target
print(X_train.shape, y_train.shape)
print('{}% outliers'.format(100 * y_train.mean()))

In [ ]:
mean, stdev = X_train.mean(axis=0), X_train.std(axis=0)

In [ ]:
X_train = (X_train - mean) / stdev

## Load or define outlier detector

The pretrained outlier and adversarial detectors used in the example notebooks can be found [here](https://console.cloud.google.com/storage/browser/seldon-models/alibi-detect). You can use the built-in ```fetch_detector``` function which saves the pre-trained models in a local directory ```filepath``` and loads the detector. Alternatively, you can train a detector from scratch:

In [ ]:
load_outlier_detector = False

In [ ]:
filepath = '/content/models/OutlierVAE'
if not os.path.exists(filepath):
  os.mkdir(filepath)

In [ ]:
# define model, initialize, train and save outlier detector
    
n_features = X_train.shape[1]
latent_dim = 2
    
encoder_net = tf.keras.Sequential(
    [
     InputLayer(input_shape=(n_features,)),
     Dense(20, activation=tf.nn.relu),
     Dense(15, activation=tf.nn.relu),
     Dense(7, activation=tf.nn.relu)
     ])

decoder_net = tf.keras.Sequential(
    [
     InputLayer(input_shape=(latent_dim,)),
     Dense(7, activation=tf.nn.relu),
     Dense(15, activation=tf.nn.relu),
     Dense(20, activation=tf.nn.relu),
     Dense(n_features, activation=None)
     ])
    
# initialize outlier detector
od = OutlierVAE(threshold=None,  # threshold for outlier score
                score_type='mse',  # use MSE of reconstruction error for outlier detection
                encoder_net=encoder_net,  # can also pass VAE model instead
                decoder_net=decoder_net,  # of separate encoder and decoder
                latent_dim=latent_dim,
                samples=5)
# train
od.fit(X_train,
       loss_fn=elbo,
       cov_elbo=dict(sim=.01),
       epochs=30,
       verbose=True)

# save the trained outlier detector
save_detector(od, filepath)

The warning tells us we still need to set the outlier threshold. This can be done with the `infer_threshold` method. We need to pass a batch of instances and specify what percentage of those we consider to be normal via `threshold_perc`. Let's assume we have some data which we know contains around 5% outliers. The percentage of outliers can be set with `perc_outlier` in the `create_outlier_batch` function.

In [ ]:
np.random.seed(0)
perc_outlier = 5
threshold_batch = create_outlier_batch(X_train, Y_train, n_samples=100, perc_outlier=perc_outlier)
X_threshold, y_threshold = threshold_batch.data.astype('float'), threshold_batch.target
X_threshold = (X_threshold - mean) / stdev
print('{}% outliers'.format(100 * y_threshold.mean()))

In [ ]:
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))

In [ ]:
save_detector(od, filepath)

## Detect outliers

We now generate a batch of data with 10% outliers and detect the outliers in the batch. 


In [ ]:
np.random.seed(1)
outlier_batch = create_outlier_batch(X_train, Y_train, n_samples=2886, perc_outlier=10)
X_outlier, y_outlier = outlier_batch.data.astype('float'), outlier_batch.target
X_outlier = (X_outlier - mean) / stdev
print(X_outlier.shape, y_outlier.shape)
print('{}% outliers'.format(100 * y_outlier.mean()))

Predict outliers:

In [ ]:
od_preds = od.predict(X_outlier,
                      outlier_type='instance',    # use 'feature' or 'instance' level
                      return_feature_score=True,  # scores used to determine outliers
                      return_instance_score=True)
print(list(od_preds['data'].keys()))

## Display results

In [ ]:
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {:.4f}'.format(f1))
cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

In [ ]:
plot_instance_score(od_preds, y_outlier, labels, od.threshold)

In [ ]:
roc_data = {'VAE': {'scores': od_preds['data']['instance_score'], 'labels': y_outlier}}
plot_roc(roc_data)

## Investigate instance level outlier

1.   List item
2.   List item



We can now take a closer look at some of the individual predictions on `X_outlier`. 

In [ ]:
X_recon = od.vae(X_outlier).numpy()  # reconstructed instances by the VAE

In [ ]:
plot_feature_outlier_tabular(od_preds,
                             X_outlier,
                             X_recon=X_recon,
                             threshold=od.threshold,
                             instance_ids=None,  # pass a list with indices of instances to display
                             max_instances=5,  # max nb of instances to display
                             top_n=5,  # only show top_n features ordered by outlier score
                             outliers_only=False,  # only show outlier predictions
                             feature_names=columns,  # add feature names
                             figsize=(20, 30))

## Drift Detection

### Method

The drift detector applies feature-wise two-sample [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) (K-S) tests for the continuous numerical features.

We split the data in a reference set and 2 test sets on which we test the data drift:

In [ ]:
n_ref = 900
n_test = 900

X_ref, X_t0, X_t1 = X_train[:n_ref], X_train[n_ref:n_ref + n_test], X_train[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

### Detect drift

Initialize the detector:

In [ ]:
cd = TabularDrift(X_ref, p_val=.05)

In [ ]:
preds = cd.predict(X_t0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
for f in range(cd.n_features):
    stat = 'K-S'
    fname = columns[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

None of the feature-level p-values are below the threshold.

In [ ]:
preds['data']['threshold']

If you are interested in individual feature-wise drift, this is also possible:

In [ ]:
fpreds = cd.predict(X_t0, drift_type='feature')

In [ ]:
for f in range(cd.n_features):
    stat = 'K-S'
    fname = columns[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

In [ ]:
preds = cd.predict(X_t1)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

We can again investigate the individual features:

In [ ]:
for f in range(cd.n_features):
    stat = 'K-S'
    fname = columns[f]
    is_drift = (preds['data']['p_val'][f] < preds['data']['threshold']).astype(int)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')